In this sample solutions you'll note that we change the timezone, normally you don't need to do this as the timezone is set by the timezone of your computer. As this notebook is hosted in the cloud on Google colab, we don't know where the machines are located, and therefore what the local timezone is. In order to make sure we get our local timezone and not the computers local timezone we use the python library `pytz`.

# Option 1:
Copy your notebook line for line into a function

In [ ]:
# Import necessary modules
import pytz
from datetime import datetime
import requests
import pandas as pd

def get_weather_loop(cities):
  # API key for openweathermap
  API_key = 'YOUR_API_KEY'

  # Set timezone to Europe/Berlin
  tz = pytz.timezone('Europe/Berlin')
  # Get current date and time
  now = datetime.now().astimezone(tz)

  # Initialize dictionary to store weather data
  weather_dict = {'city': [],
                'country': [],
                'forecast_time': [],
                'outlook': [],
                'detailed_outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'clouds': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'wind_deg': [],
                'humidity': [],
                'pressure': [],
                'information_retrieved_at': []}

  # Loop over all cities provided
  for city in cities:
    # Construct URL for the API request
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric"
    # Make the API request
    response = requests.get(url)
    # Parse the response into JSON format
    json = response.json()

    # Loop over all the forecasts returned in the JSON response
    for i in json['list']:
      # Append the relevant pieces of data to the appropriate list in the weather_dict dictionary
      weather_dict['city'].append(json['city']['name'])
      weather_dict['country'].append(json['city']['country'])
      weather_dict['forecast_time'].append(i['dt_txt'])
      weather_dict['outlook'].append(i['weather'][0]['main'])
      weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
      weather_dict['temperature'].append(i['main']['temp'])
      weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
      weather_dict['clouds'].append(i['clouds']['all'])
      # Check if rain and snow are in the forecast, otherwise append '0'
      weather_dict['rain'].append(i['rain']['3h']) if 'rain' in i else weather_dict['rain'].append('0')
      weather_dict['snow'].append(i['snow']['3h']) if 'snow' in i else weather_dict['snow'].append('0')
      weather_dict['wind_speed'].append(i['wind']['speed'])
      weather_dict['wind_deg'].append(i['wind']['deg'])
      weather_dict['humidity'].append(i['main']['humidity'])
      weather_dict['pressure'].append(i['main']['pressure'])
      # Append the date and time that the information was retrieved
      weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))

  # Convert the weather_dict into a pandas DataFrame and return it
  return pd.DataFrame(weather_dict)

In [ ]:
cities = ["Berlin", "Hamburg", "Leipzig", "London"]

In [ ]:
get_weather_loop(cities)

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,2023-07-04 12:00:00,Clear,clear sky,20.80,20.22,0,0,0,5.49,254,49,1012,04/07/2023 13:08:32
1,Berlin,DE,2023-07-04 15:00:00,Clouds,scattered clouds,21.05,20.44,29,0,0,4.90,266,47,1012,04/07/2023 13:08:32
2,Berlin,DE,2023-07-04 18:00:00,Clouds,scattered clouds,20.48,19.81,48,0,0,3.82,288,47,1013,04/07/2023 13:08:32
3,Berlin,DE,2023-07-04 21:00:00,Clouds,scattered clouds,16.37,15.60,48,0,0,1.09,303,59,1014,04/07/2023 13:08:32
4,Berlin,DE,2023-07-05 00:00:00,Clouds,broken clouds,15.94,15.16,64,0,0,1.95,136,60,1014,04/07/2023 13:08:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,London,GB,2023-07-08 21:00:00,Clouds,overcast clouds,18.46,17.72,100,0,0,3.03,239,52,1018,04/07/2023 13:08:32
156,London,GB,2023-07-09 00:00:00,Clouds,overcast clouds,15.95,15.27,95,0,0,2.74,229,64,1019,04/07/2023 13:08:32
157,London,GB,2023-07-09 03:00:00,Clouds,overcast clouds,14.31,13.68,97,0,0,2.67,227,72,1019,04/07/2023 13:08:32
158,London,GB,2023-07-09 06:00:00,Clouds,overcast clouds,15.03,14.39,98,0,0,2.42,235,69,1019,04/07/2023 13:08:32


# Option 2:
Improve your function - we not only improved the structure and code, but this time we also include country for extra compexity, but better accuracy (afterall, there are a lot of London's and Berlin's in the world).

In [ ]:
# Import necessary modules
import pandas as pd
import requests
from datetime import datetime
import pytz

# Define function to get a city's weather data
def get_weather_data(city, country, API_key):
    # Construct request URL for OpenWeatherMap API
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city},{country}&appid={API_key}&units=metric"

    # Send the request and get the response
    response = requests.get(url)

    # Parse the response as JSON
    json = response.json()

    # Initialize an empty list to hold the weather data
    forecast_data = []

    # Extract city and country info from the JSON data
    city_name = json['city']['name']
    country_code = json['city']['country']

    # Iterate through the forecast data in the JSON response
    for i in json['list']:
        # Get the current time in Berlin
        tz = pytz.timezone('Europe/Berlin')
        now = datetime.now().astimezone(tz).strftime("%d/%m/%Y %H:%M:%S")

        # Extract rain and snow data if available
        rain = i['rain']['3h'] if 'rain' in i and '3h' in i['rain'] else '0'
        snow = i['snow']['3h'] if 'snow' in i and '3h' in i['snow'] else '0'

        # Build dictionary of weather data
        forecast = {
            'information_retrieved_at': now,
            'city': city_name,
            'country': country_code,
            'forecast_time': i['dt_txt'],
            'outlook': i['weather'][0]['main'],
            'detailed_outlook': i['weather'][0]['description'],
            'temperature': i['main']['temp'],
            'temperature_feels_like': i['main']['feels_like'],
            'clouds': i['clouds']['all'],
            'rain': rain,
            'snow': snow,
            'wind_speed': i['wind']['speed'],
            'wind_deg': i['wind']['deg'],
            'humidity': i['main']['humidity'],
            'pressure': i['main']['pressure']
        }

        # Append forecast data to list
        forecast_data.append(forecast)

    # Return list of weather data
    return forecast_data

# Define function to iterate through all specified cities and get weather data
def get_weather_loop(cities_df):
    # Specify API key
    API_key = 'YOUR_API_KEY'

    # Initialize list to hold all weather data dictionaries
    all_weather_data = []

    # Loop through dataframe of cities
    for _, row in cities_df.iterrows():
        # Extract city and country from the row
        city = row['City']
        country = row['Country']

        # Call the get_weather_data function and append its results to the all_weather_data list
        city_weather_data = get_weather_data(city, country, API_key)
        all_weather_data.extend(city_weather_data)

    # Create a dataframe from the weather data and return it
    df = pd.DataFrame(all_weather_data)
    return df

In [ ]:
cities_dict = {"City": ["Berlin", "Hamburg", "Leipzig", "London"],
              "Country": ["DE", "DE", "DE", "UK"]}

cities_df = pd.DataFrame(cities_dict)

cities_df

,City,Country
0,Berlin,DE
1,Hamburg,DE
2,Leipzig,DE
3,London,UK


In [ ]:
get_weather_loop(cities_df)

,information_retrieved_at,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure
0,04/07/2023 13:08:33,Berlin,DE,2023-07-04 12:00:00,Clear,clear sky,21.21,20.80,0,0,0,5.49,254,54,999
1,04/07/2023 13:08:33,Berlin,DE,2023-07-04 15:00:00,Clouds,scattered clouds,21.32,20.81,29,0,0,4.90,266,50,1004
2,04/07/2023 13:08:33,Berlin,DE,2023-07-04 18:00:00,Clouds,scattered clouds,20.62,20.02,48,0,0,3.82,288,49,1008
3,04/07/2023 13:08:33,Berlin,DE,2023-07-04 21:00:00,Clouds,scattered clouds,16.37,15.60,48,0,0,1.09,303,59,1014
4,04/07/2023 13:08:33,Berlin,DE,2023-07-05 00:00:00,Clouds,broken clouds,15.94,15.16,64,0,0,1.95,136,60,1014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,04/07/2023 13:08:33,London,GB,2023-07-08 21:00:00,Clouds,overcast clouds,18.46,17.72,100,0,0,3.03,239,52,1018
156,04/07/2023 13:08:33,London,GB,2023-07-09 00:00:00,Clouds,overcast clouds,15.95,15.27,95,0,0,2.74,229,64,1019
157,04/07/2023 13:08:33,London,GB,2023-07-09 03:00:00,Clouds,overcast clouds,14.31,13.68,97,0,0,2.67,227,72,1019
158,04/07/2023 13:08:33,London,GB,2023-07-09 06:00:00,Clouds,overcast clouds,15.03,14.39,98,0,0,2.42,235,69,1019
